Observable Trends:
1. It appears that Charter schools tend to have a higher average score in all aspects when compared to District schools. More analysis needs to be done to find out why; if the higher averages are a result of how the school educates their students or if it is because of a smaller teacher to student ratio. There were only 15 schools used so a larger collection of data would yield better results.
2. The school size did seem to matter with the smaller schools having higher scores which is more evident in the math scores. The larger schools were found more in the bottom five schools when compared to the schools in the top five of the data.
3. The schools seem to do well regardless of their budget with scores similar throughout the math and reading scores.

In [17]:
import pandas as pd
import csv

#Retrieve files
school_data = "schools_complete.csv"
student_data = "students_complete.csv"

#Read files
school_df = pd.read_csv(school_data, encoding="iso-8859-1", low_memory=False)
student_df = pd.read_csv(student_data, encoding="iso-8859-1", low_memory=False)

school_df.columns = ["School ID", "School Name", "type","size","budget"]
student_df.columns = ["Student ID", "Student Name", "gender","grade","School Name", "Reading Score", "Math Score"]

#Merge dfs 

compiled_data = pd.merge(school_df, student_df, on="School Name")

compiled_data.head()

,School ID,School Name,type,size,budget,Student ID,Student Name,gender,grade,Reading Score,Math Score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


District Summary

In [18]:
#Total Schools
total_schools = len(compiled_data["School Name"].unique())
print(total_schools)
#Total Students
total_students = compiled_data["Student ID"].count()
print(total_students)
#Total Budget
total_budget = sum(school_df["budget"])
print(total_budget)
#Average Math Score
avg_math = round(compiled_data["Math Score"].mean(),2)
print(avg_math)
#Average Reading Score
avg_reading = round(compiled_data["Reading Score"].mean(),2)
print(avg_reading)


15
39170
24649428
78.99
81.88


In [19]:
#students that pass
pass_math = compiled_data.loc[compiled_data["Math Score"] > 70,:] 
percentage_pass_math = len(pass_math) / len(compiled_data)                  
print(percentage_pass_math)

pass_reading = compiled_data.loc[compiled_data["Reading Score"] > 70,:]
percentage_pass_reading = len(pass_reading) / len(compiled_data)
print(percentage_pass_reading)

passing_students = (percentage_pass_math + percentage_pass_reading) / 2
passing_students

0.7239213683941792
0.8297166198621394


0.7768189941281594

In [21]:
district_summary = pd.DataFrame({"Total Schools": [total_schools], 
                                 "Total Students": [total_students], 
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [avg_math], 
                                 "Average Reading Score": [avg_reading],
                                 "% Passing Math": [percentage_pass_math],
                                 "% Passing Reading": [percentage_pass_reading],
                                 "% Overall Passing Rate": [passing_students]})

district_summary = district_summary[["Total Schools", "Total Students", "Total Budget",
                                     "Average Math Score", 
                                     "Average Reading Score",
                                     "% Passing Math",
                                     "% Passing Reading",
                                     "% Overall Passing Rate"]]

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the data frame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,0.723921,0.829717,0.776819


School Summary

In [34]:
# Determine the School Type
school_types = school_df.set_index(["School Name"])["type"]

# Calculate the total student count
per_school_counts = compiled_data["School Name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = compiled_data.groupby(["School Name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores
per_school_math = compiled_data.groupby(["School Name"]).mean()["Math Score"]
per_school_reading = compiled_data.groupby(["School Name"]).mean()["Reading Score"]

# Calculate the passing scores by creating a filtered data frame
school_passing_math = compiled_data[(compiled_data["Math Score"] > 70)]
school_passing_reading = compiled_data[(compiled_data["Reading Score"] > 70)]

per_school_passing_math = school_passing_math.groupby(["School Name"]).count()["Student Name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["School Name"]).count()["Student Name"] / per_school_counts * 100
overall_passing_rate = (per_school_passing_math + per_school_passing_reading) / 2

# Convert to data frame
per_school_summary = pd.DataFrame({"School Type": school_types,
                                   "Total Students": per_school_counts,
                                   "Total School Budget": per_school_budget,
                                   "Per Student Budget": per_school_capita,
                                   "Average Math Score": per_school_math,
                                   "Average Reading Score": per_school_reading,
                                   "% Passing Math": per_school_passing_math,
                                   "% Passing Reading": per_school_passing_reading,
                                   "% Overall Passing Rate": overall_passing_rate})

# Minor data munging
per_school_summary = per_school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                         "Average Math Score", "Average Reading Score", 
                                         "% Passing Math", "% Passing Reading", 
                                         "% Overall Passing Rate"]]
per_school_summary["Total School Budget"] = per_school_summary["Total Students"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$4,976.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
Cabrera High School,Charter,1858,"$1,858.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Figueroa High School,District,2949,"$2,949.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Ford High School,District,2739,"$2,739.00",$644.00,77.102592,80.746258,65.753925,77.510040,71.631982
Griffin High School,Charter,1468,"$1,468.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134
Hernandez High School,District,4635,"$4,635.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
Holden High School,Charter,427,$427.00,$581.00,83.803279,83.814988,90.632319,92.740047,91.686183
Huang High School,District,2917,"$2,917.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$4,761.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
Pena High School,Charter,962,$962.00,$609.00,83.839917,84.044699,91.683992,92.203742,91.943867


Top Performing Schools(Passing Rate)

In [35]:
# Sort and show top five schools
top_schools = per_school_summary.sort_values(["% Overall Passing Rate"], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Wilson High School,Charter,2283,"$2,283.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,$962.00,$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,1800,"$1,800.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,1858,"$1,858.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,$427.00,$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


In [ ]:
Low Performing Schools(Passing Rate)

In [36]:
# Sort and show bottom five schools
bottom_schools = per_school_summary.sort_values(["% Overall Passing Rate"], ascending=True)
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$3,999.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,2917,"$2,917.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,4761,"$4,761.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,2949,"$2,949.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,4635,"$4,635.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


Scores by School Type

In [37]:
type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
overall_passing_rate = (type_passing_math + type_passing_reading) / 2

type_summary = pd.DataFrame({"Average Math Score" : type_math_scores,
                             "Average Reading Score": type_reading_scores,
                             "% Passing Math": type_passing_math,
                             "% Passing Reading": type_passing_reading,
                             "% Overall Passing Rate": overall_passing_rate})

type_summary = type_summary[["Average Math Score", 
                             "Average Reading Score",
                             "% Passing Math",
                             "% Passing Reading",
                             "% Overall Passing Rate"]]

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
